This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to rotation splines](index.ipynb)

# Barry--Goldman Algorithm

We can try to use the
[Barry--Goldman algorithm for non-uniform Euclidean Catmull--Rom splines](../euclidean/catmull-rom-barry-goldman.ipynb)
using [Slerp](slerp.ipynb) instead of linear interpolations,
just as we have done with [De Casteljau's algorithm](de-casteljau.ipynb).

In [ ]:
def slerp(one, two, t):
    return (two * one.inverse())**t * one

In [ ]:
def barry_goldman(rotations, times, t):
    q0, q1, q2, q3 = rotations
    t0, t1, t2, t3 = times
    return slerp(
        slerp(
            slerp(q0, q1, (t - t0) / (t1 - t0)),
            slerp(q1, q2, (t - t1) / (t2 - t1)),
            (t - t0) / (t2 - t0)),
        slerp(
            slerp(q1, q2, (t - t1) / (t2 - t1)),
            slerp(q2, q3, (t - t2) / (t3 - t2)),
            (t - t1) / (t3 - t1)),
        (t - t1) / (t2 - t1))

Example:

In [ ]:
import numpy as np

[helper.py](helper.py)

In [ ]:
from helper import angles2quat, animate_rotations, display_animation

In [ ]:
q0 = angles2quat(0, 0, 0)
q1 = angles2quat(90, 0, 0)
q2 = angles2quat(90, 90, 0)
q3 = angles2quat(90, 90, 90)

In [ ]:
t0 = 0
t1 = 1
t2 = 3
t3 = 3.5

In [ ]:
frames = 50

In [ ]:
ani = animate_rotations({
    'Barry–Goldman (q0, q1, q2, q3)': [
        barry_goldman([q0, q1, q2, q3], [t0, t1, t2, t3], t)
        for t in np.linspace(t1, t2, frames)
    ],
    'Slerp (q1, q2)': slerp(q1, q2, np.linspace(0, 1, frames)),
}, figsize=(6, 2))

In [ ]:
display_animation(ani, default_mode='once')

[splines.quaternion.BarryGoldman](../python-module/splines.quaternion.rst#splines.quaternion.BarryGoldman) class

In [ ]:
from splines.quaternion import BarryGoldman

In [ ]:
import numpy as np

[helper.py](helper.py)

In [ ]:
from helper import angles2quat, animate_rotations, display_animation

In [ ]:
rotations = [
    angles2quat(0, 0, 180),
    angles2quat(0, 45, 90),
    angles2quat(90, 45, 0),
    angles2quat(90, 90, -90),
    angles2quat(180, 0, -180),
    angles2quat(-90, -45, 180),
]

In [ ]:
grid = np.array([0, 0.5, 2, 5, 6, 7, 9])

In [ ]:
bg = BarryGoldman(rotations, grid)

For comparison ... [Catmull--Rom-like quaternion spline](catmull-rom-non-uniform.ipynb)

[splines.quaternion.CatmullRom](../python-module/splines.quaternion.rst#splines.quaternion.CatmullRom) class

In [ ]:
from splines.quaternion import CatmullRom

In [ ]:
cr = CatmullRom(rotations, grid, endconditions='closed')

In [ ]:
def evaluate(spline, samples=200):
    times = np.linspace(spline.grid[0], spline.grid[-1], samples, endpoint=False)
    return spline.evaluate(times)

In [ ]:
ani = animate_rotations({
    'Barry–Goldman': evaluate(bg),
    'Catmull–Rom-like': evaluate(cr),
}, figsize=(4, 2))

In [ ]:
display_animation(ani, default_mode='loop')

In [ ]:
rotations = [
    angles2quat(90, 0, -45),
    angles2quat(179, 0, 0),
    angles2quat(181, 0, 0),
    angles2quat(270, 0, -45),
    angles2quat(0, 90, 90),
]

In [ ]:
s_uniform = BarryGoldman(rotations)
s_chordal = BarryGoldman(rotations, alpha=1)
s_centripetal = BarryGoldman(rotations, alpha=0.5)

In [ ]:
ani = animate_rotations({
    'uniform': evaluate(s_uniform, samples=300),
    'chordal': evaluate(s_chordal, samples=300),
    'centripetal': evaluate(s_centripetal, samples=300),
}, figsize=(6, 2))

In [ ]:
display_animation(ani, default_mode='loop')

## Constant Angular Speed

Not very efficient,
De Casteljau's algorithm is faster because it directly provides the tangent.

In [ ]:
from splines import ConstantSpeedAdapter

In [ ]:
class BarryGoldmanWithDerivative(BarryGoldman):
    
    delta_t = 0.000001
    
    def evaluate(self, t, n=0):
        """Evaluate quaternion or angular velocity."""
        if not np.isscalar(t):
            return np.array([self.evaluate(t, n) for t in t])
        if n == 0:
            return super().evaluate(t)
        elif n == 1:
            # NB: We move the interval around because
            #     we cannot access times before and after
            #     the first and last time, respectively.
            fraction = (t - self.grid[0]) / (self.grid[-1] - self.grid[0])
            before = super().evaluate(t - fraction * self.delta_t)
            after = super().evaluate(t + (1 - fraction) * self.delta_t)
            # NB: Double angle
            return (after * before.inverse()).log_map() * 2 / self.delta_t
        else:
            raise ValueError('Unsupported n: {!r}'.format(n))

In [ ]:
s = ConstantSpeedAdapter(BarryGoldmanWithDerivative(rotations, alpha=0.5))

Takes a long time!

In [ ]:
ani = animate_rotations({
    'non-constant speed': evaluate(s_centripetal),
    'constant speed': evaluate(s),
}, figsize=(4, 2))

In [ ]:
display_animation(ani, default_mode='loop')